In [1]:
import torchaudio
torchaudio.set_audio_backend("soundfile")

from datasets.Sampler import Sampler
from graphs.models.SpeakerNet import SpeakerNet
from graphs.models.VGGVox import VGGVox

from agents.trainer import Trainer


from utils.config import *

#JSON = 'configs/VGGVox.json'
JSON = 'configs/ResNetSE34L.json'
#JSON = 'configs/Test.json'

config = process_config(JSON)

C:\Users\giang\anaconda3\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')
[INFO]: Experiment: ResNetSE34L.


In [2]:
trainer = Trainer(config)

[INFO]: Operation will be on ***** GPU-CUDA ***** 
[INFO]: __Python VERSION:  3.6.13 (default, Feb 19 2021, 05:17:09) [MSC v.1916 64 bit (AMD64)]
[INFO]: __pyTorch VERSION:  1.8.1+cu102
[INFO]: __CUDA VERSION
[INFO]: __CUDNN VERSION:  7605
[INFO]: __Number CUDA Devices:  1
[INFO]: __Devices
[INFO]: Active CUDA Device: GPU 0
[INFO]: Available devices  1
[INFO]: Current cuda device  0
[INFO]: Loading checkpoint 'experiments\ResNetSE34L\checkpoints/'
[INFO]: No checkpoint exists from 'experiments\ResNetSE34L\checkpoints/'. Skipping...


In [3]:
trainer.run()

Epoch-1 Validation | VEER 2.1792 MDC 0.16909| Efficiency 98.60 : 100%|██████████| 37720/37720 [43:55<00:00, 14.31it/s]
